In [20]:
from google.colab import files
uploaded = files.upload()

Saving Future_Interns_Ass_1_Superstone_Dataset.xlsx to Future_Interns_Ass_1_Superstone_Dataset (2).xlsx


In [21]:
import pandas as pd

df = pd.read_excel("Future_Interns_Ass_1_Superstone_Dataset.xlsx")
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,2016-08-11 00:00:00,2016-11-11 00:00:00,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,2016-08-11 00:00:00,2016-11-11 00:00:00,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,2016-12-06 00:00:00,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,2015-11-10 00:00:00,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,2015-11-10 00:00:00,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164


In [22]:
df.info()
df.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   object 
 2   Order Date     9994 non-null   object 
 3   Ship Date      9994 non-null   object 
 4   Ship Mode      9994 non-null   object 
 5   Customer ID    9994 non-null   object 
 6   Customer Name  9994 non-null   object 
 7   Segment        9994 non-null   object 
 8   Country        9994 non-null   object 
 9   City           9994 non-null   object 
 10  State          9994 non-null   object 
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   object 
 13  Product ID     9994 non-null   object 
 14  Category       9994 non-null   object 
 15  Sub-Category   9994 non-null   object 
 16  Product Name   9994 non-null   object 
 17  Sales          9994 non-null   float64
 18  Quantity

,0
Row ID,0
Order ID,0
Order Date,0
Ship Date,0
Ship Mode,0
Customer ID,0
Customer Name,0
Segment,0
Country,0
City,0


Fix Date Columns

In [23]:
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])

Clean Sales Column

In [24]:
df['Sales'] = (
    df['Sales']
    .astype(str)
    .str.replace(',', '')
    .astype(float)
)

Remove duplicates

In [25]:
df = df.drop_duplicates(subset=['Order ID', 'Product ID'])

In [26]:
df['Profit'] = df['Profit'].fillna(0)
df = df.dropna(subset=['Order Date', 'Sales'])

Handle missing values

In [27]:
df['Profit'] = df['Profit'].fillna(0)
df = df.dropna(subset=['Order Date', 'Sales'])

#Aggregate Data for Time Series Forecasting

**Time-series models must have one row per time unit.**

Set Order Date as index

In [28]:
if 'Order Date' in df.columns:
    df = df.set_index('Order Date')

Monthly Sales Aggregation

In [29]:
monthly_sales = df.resample('M')['Sales'].sum().reset_index()
monthly_sales.columns = ['ds', 'y']  # Required for Prophet
monthly_sales.head()

/tmp/ipython-input-466292160.py:1: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_sales = df.resample('M')['Sales'].sum().reset_index()


,ds,y
0,2014-01-31,28953.706
1,2014-02-28,12743.108
2,2014-03-31,54801.906
3,2014-04-30,24428.644
4,2014-05-31,29639.834


#Feature Engineering

Monthly Average Sales

In [30]:
monthly_sales['monthly_avg'] = monthly_sales['y'].rolling(3).mean()

Seasonal Indicators

In [31]:
monthly_sales['month'] = monthly_sales['ds'].dt.month
monthly_sales['year'] = monthly_sales['ds'].dt.year

Holiday Spike Indicator

In [32]:
monthly_sales['holiday_spike'] = monthly_sales['month'].isin([10, 11, 12]).astype(int)

#Train a Time Series Forecasting Mode

using Prophet

In [33]:
!pip install prophet
from prophet import Prophet

model = Prophet()
model.fit(monthly_sales[['ds','y']])

future = model.make_future_dataframe(periods=12, freq='M')
forecast = model.predict(future)

forecast[['ds','yhat','yhat_lower','yhat_upper']].tail()


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
/usr/local/lib/python3.12/dist-packages/prophet/forecaster.py:1872: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  dates = pd.date_range(


,ds,yhat,yhat_lower,yhat_upper
55,2018-08-31,70875.024026,60064.485266,80913.778148
56,2018-09-30,85359.001704,75916.108390,95132.689840
57,2018-10-31,62041.706836,52109.740436,72202.204024
58,2018-11-30,85398.830293,75713.888796,96364.502207
59,2018-12-31,81676.191647,71919.781643,91950.528546


XGBoost (Advanced, Resume-Boosting)

In [34]:
!pip install xgboost
from xgboost import XGBRegressor

X = monthly_sales[['month','year','holiday_spike']]
y = monthly_sales['y']

model = XGBRegressor()
model.fit(X, y)


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [35]:
forecast.to_csv('sales_forecast.csv', index=False)
files.download("sales_forecast.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>